# Asistente Turístico de Tenerife
**Práctica Final - Large Language Models**

In [1]:
# Section 1: Setup and PDF Loading
from src.conf import OPENAI_API_KEY, MODEL_CONFIG, PDF_PATH
from src.logger import logger
from src.api_client import OpenAIClient
from src.data_loader import DataLoader

print(f"API Key loaded: {OPENAI_API_KEY[:8]}...")
print(f"Model config: {MODEL_CONFIG}")

API Key loaded: sk-proj-...
Model config: {'model': 'gpt-4o-mini', 'temperature': 0.3, 'max_tokens': 1024, 'top_p': 0.9}


In [2]:
# Initialize OpenAI client
client = OpenAIClient()

if client.test_connection():
    print("✅ OpenAI connection OK")
else:
    print("❌ OpenAI connection failed")

2026-01-21 21:15:10,860 - INFO - OpenAI client initialized with model: gpt-4o-mini
2026-01-21 21:15:13,594 - INFO - OpenAI connection test successful


✅ OpenAI connection OK


In [3]:
# Load PDF
loader = DataLoader(PDF_PATH)
pages = loader.load()
stats = loader.get_stats()
print(f"✅ PDF loaded: {stats}")

2026-01-21 21:15:15,723 - INFO - DataLoader initialized with: data/TENERIFE.pdf
2026-01-21 21:15:16,038 - INFO - Loaded 25 pages from data/TENERIFE.pdf


✅ PDF loaded: {'num_pages': 25, 'total_chars': 16094, 'total_words': 2817, 'avg_chars_per_page': 643}


In [4]:
# Section 2: Vector Store
from src.vector_store import VectorStore

vector_store = VectorStore()
num_chunks = vector_store.build_from_documents(pages)
print(f"✅ Vector store built: {num_chunks} chunks")
print(f"Chunk stats: {vector_store.get_chunk_stats()}")

2026-01-21 21:15:20,633 - INFO - VectorStore initialized (chunk_size=1000, overlap=200)
2026-01-21 21:15:20,634 - INFO - Created 30 chunks from 25 pages
2026-01-21 21:15:21,878 - INFO - Vector store built and persisted to ./chroma_db


✅ Vector store built: 30 chunks
Chunk stats: {'num_chunks': 30, 'avg_chunk_size': 570, 'min_chunk_size': 86, 'max_chunk_size': 987}


In [5]:
# DEBUG: Test raw retrieval to see what chunks are returned
query = "¿Qué playas hay en el norte de Tenerife?"
results = vector_store.search(query, k=5)

print(f"Query: {query}\n")
print(f"Retrieved {len(results)} chunks:\n")
for i, doc in enumerate(results):
    page = doc.metadata.get('page', 'N/A')
    print(f"--- Chunk {i+1} (page {page}) ---")
    print(doc.page_content[:400])
    print()

2026-01-21 21:15:25,982 - INFO - Search query: '¿Qué playas hay en el norte de Tenerife?...' returned 5 results


Query: ¿Qué playas hay en el norte de Tenerife?

Retrieved 5 chunks:

--- Chunk 1 (page 22) ---
• El Médano / La Tejita 
Playas tipiquísimas de Tenerife y otra de las estampas de la isla con Montaña Roja al 
fondo. Nota: La Tejita es playa nudista para que lo tengáis en cuenta.  
 
 
• Los Cristianos 
De la zona de Los Cristianos que me venga a la mente os recomendaría la Playa de 
Los Cristianos o la Playa de Las Vistas y creo que poco más (la imagen es la Playa de 
Las Vistas).

--- Chunk 2 (page 0) ---
TENERIFE – LUGARES DE INTERÉS 
SITIOS QUE VER 
 
ZONA NORTE 
 
• Santa Cruz de Tenerife: 
Santa Cruz de Tenerife es la capital de la isla. Quizás la ruta a seguir si vais a Santa 
Cruz sería: 
- Aparcar en el aparcamiento del Parque Marítimo (ubicación). 
- Caminar por la Avenida Marítima hasta Plaza de España (ubicación). 
- Por el camino de la Avenida Marítima, ver el auditorio de Tenerife (ubic

--- Chunk 3 (page 10) ---
tendréis poca playa (especialmente a la del Ancón) y tened mu

In [6]:
# DEBUG: Try a more specific query
query = "playas arena negra norte Tenerife Benijo Bollullo"
results = vector_store.search(query, k=5)

print(f"Query: {query}\n")
for i, doc in enumerate(results):
    page = doc.metadata.get('page', 'N/A')
    print(f"--- Chunk {i+1} (page {page}) ---")
    print(doc.page_content[:400])
    print()

2026-01-21 21:16:07,554 - INFO - Search query: 'playas arena negra norte Tenerife Benijo Bollullo...' returned 5 results


Query: playas arena negra norte Tenerife Benijo Bollullo

--- Chunk 1 (page 7) ---
o Playa de Benijo [vídeo - ubicación] 
Playa de arena negra, si el día está despejado, de los atardeceres más 
espectaculares que podéis ver en la isla. 
Para llegar a esta playa, aparcar por aquí y, para acceder a la playa, se accede 
desde aquí (puede ser que os encontréis con el acceso cortado con una valla, 
pero da igual, podéis acceder igualmente). 
• La Orotava 
Este es el pueblo más bonito

--- Chunk 2 (page 16) ---
o Playa de Troya 
 
 
Encima de esta playa se encuentra el Monkey Beach Club, uno de los beach 
clubs más populares de Tenerife. Tiene varias zonas, BésameMucho – zona 
más tipo restaurante -, Kong – zona tipo cocktails -, y Cielo – también zona 
tipo cocktails pero en la parte de arriba y que mola mucho para tomaros 
algún cocktail mientras veis el atardecer. También abre como beach club de 
tard

--- Chunk 3 (page 1) ---
o Auditorio de Tenerife [vídeo - ubicación] 
 
 
o Plaza de Es

In [7]:
# Section 3: RAG Chain with Multi-turn Support
from src.rag_chain import RAGChain

# Increase k to retrieve more chunks for better context
rag = RAGChain(client, vector_store, max_history=5)
print("✅ RAG Chain initialized with multi-turn support")

2026-01-21 21:16:14,693 - INFO - RAGChain initialized (max_history=5)


✅ RAG Chain initialized with multi-turn support


In [8]:
# Test with more chunks (k=5)
result = rag.query("¿Qué playas hay en el norte de Tenerife?", k=5)

print("PREGUNTA:", result["question"])
print("\nRESPUESTA:")
print(result["answer"])
print("\nFUENTES:")
for s in result["sources"]:
    print(f"  - Página {s['page']}")

2026-01-21 21:16:22,099 - INFO - Search query: '¿Qué playas hay en el norte de Tenerife?...' returned 5 results
2026-01-21 21:16:22,101 - INFO - Retrieved 5 chunks for: ¿Qué playas hay en el norte de Tenerife?...
2026-01-21 21:16:25,978 - INFO - Completion received - tokens used: 1342
2026-01-21 21:16:25,979 - INFO - Generated answer of 553 chars


PREGUNTA: ¿Qué playas hay en el norte de Tenerife?

RESPUESTA:
En el norte de Tenerife, las playas que se destacan son:

1. **Playa Martiánez**: Ubicada en el Puerto de La Cruz, es una playa que forma parte de un conjunto de piscinas naturales.
   
2. **Playa de Las Teresitas**: Aunque se menciona en el contexto de Santa Cruz de Tenerife, es una playa popular que se encuentra en la zona.

Recuerda que las playas en el norte suelen ser de arena negra, a diferencia de las del sur que son de arena blanca. Si necesitas más información sobre alguna playa específica o actividades en la zona, ¡no dudes en preguntar!

FUENTES:
  - Página 22
  - Página 0
  - Página 10
  - Página 10
  - Página 15


In [9]:
# Section 4: Multi-turn Conversation Demo
rag.clear_history()
print("Historia limpiada. Iniciando conversación multi-turno...\n")
print("=" * 60)

2026-01-21 21:16:33,368 - INFO - Conversation history cleared


Historia limpiada. Iniciando conversación multi-turno...



In [10]:
# Turn 1: Ask about beaches (with more chunks)
result = rag.query("¿Qué playas hay en el norte de Tenerife?", k=5)
print("👤 Usuario:", result["question"])
print("\n🤖 Asistente:", result["answer"])
print("\n" + "=" * 60)

2026-01-21 21:16:36,634 - INFO - Search query: '¿Qué playas hay en el norte de Tenerife?...' returned 5 results
2026-01-21 21:16:36,635 - INFO - Retrieved 5 chunks for: ¿Qué playas hay en el norte de Tenerife?...
2026-01-21 21:16:39,290 - INFO - Completion received - tokens used: 1328
2026-01-21 21:16:39,291 - INFO - Generated answer of 476 chars


👤 Usuario: ¿Qué playas hay en el norte de Tenerife?

🤖 Asistente: En el norte de Tenerife, las playas recomendadas son:

1. **Playa Martiánez**: Ubicada en Puerto de La Cruz, es conocida por su entorno y cercanía a diversas atracciones turísticas.
2. **Playa de Las Teresitas**: Aunque se menciona en el contexto de Santa Cruz, es una playa que se puede visitar fácilmente desde la capital.

Recuerda que las playas en el norte suelen tener arena negra, a diferencia de las del sur que son de arena blanca. ¡Espero que disfrutes de tu visita!



In [11]:
# Turn 2: Follow-up question
result = rag.query("¿Cuál de esas me recomiendas para ver el atardecer?", k=5)
print("👤 Usuario:", result["question"])
print("\n🤖 Asistente:", result["answer"])
print("\n" + "=" * 60)

2026-01-21 21:16:51,814 - INFO - Search query: '¿Cuál de esas me recomiendas para ver el atardecer...' returned 5 results
2026-01-21 21:16:51,816 - INFO - Retrieved 5 chunks for: ¿Cuál de esas me recomiendas para ver el atardecer...
2026-01-21 21:16:55,552 - INFO - Completion received - tokens used: 906
2026-01-21 21:16:55,553 - INFO - Generated answer of 370 chars


👤 Usuario: ¿Cuál de esas me recomiendas para ver el atardecer?

🤖 Asistente: Te recomiendo la **Playa de Benijo** para disfrutar de uno de los atardeceres más espectaculares de la isla, especialmente si el día está despejado. Esta playa de arena negra ofrece un entorno impresionante para contemplar el ocaso. Recuerda que para acceder, puedes encontrar el acceso cortado con una valla, pero aún así puedes llegar sin problemas. ¡No te lo pierdas!



In [12]:
# Turn 3: Another follow-up
result = rag.query("¿Cómo llego allí?", k=5)
print("👤 Usuario:", result["question"])
print("\n🤖 Asistente:", result["answer"])
print("\n" + "=" * 60)

2026-01-21 21:17:02,671 - INFO - Search query: '¿Cómo llego allí?...' returned 5 results
2026-01-21 21:17:02,673 - INFO - Retrieved 5 chunks for: ¿Cómo llego allí?...
2026-01-21 21:17:08,066 - INFO - Completion received - tokens used: 1310
2026-01-21 21:17:08,067 - INFO - Generated answer of 583 chars


👤 Usuario: ¿Cómo llego allí?

🤖 Asistente: Para llegar a **Punta de Teno**, debes consultar los horarios y accesos, ya que actualmente solo se puede ir en guagua (línea 369 de Titsa) desde la Estación de Buenavista. 

Si decides visitar **Buenavista**, puedes aparcar cerca de la Plaza de los Remedios y disfrutar de la zona. Desde allí, puedes tomar la guagua hacia Punta de Teno.

Si te diriges a **Garachico**, puedes aparcar cerca del Castillo de San Miguel y explorar las Piscinas Naturales del Caletón. 

Recuerda verificar los horarios de las guaguas y disfrutar de tu visita. ¡Espero que tengas un gran día explorando!



In [13]:
# Check conversation history
print("📜 Historial de conversación:")
for i, msg in enumerate(rag.get_history()):
    role = "👤 Usuario" if msg["role"] == "user" else "🤖 Asistente"
    content = msg["content"][:100] + "..." if len(msg["content"]) > 100 else msg["content"]
    print(f"{i+1}. {role}: {content}")

📜 Historial de conversación:
1. 👤 Usuario: ¿Qué playas hay en el norte de Tenerife?
2. 🤖 Asistente: En el norte de Tenerife, las playas recomendadas son:

1. **Playa Martiánez**: Ubicada en Puerto de ...
3. 👤 Usuario: ¿Cuál de esas me recomiendas para ver el atardecer?
4. 🤖 Asistente: Te recomiendo la **Playa de Benijo** para disfrutar de uno de los atardeceres más espectaculares de ...
5. 👤 Usuario: ¿Cómo llego allí?
6. 🤖 Asistente: Para llegar a **Punta de Teno**, debes consultar los horarios y accesos, ya que actualmente solo se ...
